In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\aanas\AppData\Local\Temp\ipykernel_18936\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab. Natural Lenguage Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [6]:
## Read Data for the Fraudulent Email Kaggle Challenge
data_train = pd.read_csv("data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speead up development. 
# Modify for final system
data_train = data_train.head(1000)
print(data_train.shape)
data_train.fillna("",inplace=True)


data_train.head()

(1000, 2)


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


### Let's divide the training and test set into two partitions

In [7]:
# Your code
#I'll use training csv and test csv

## Data Preprocessing

In [8]:
import string
import nltk 

nltk.download('stopwords')
from nltk.corpus import stopwords

print(string.punctuation)
print(stopwords.words("english")[100:110])

from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

import re 

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aanas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aanas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [9]:
# Your code

def clean_html (html):
    """Copied from NLTK package.
    Remove HTML markup from the given string.

    :param html: the HTML string to be cleaned
    :type html: str
    :rtype: str"""
    
    # First we remove inline JavaScript/CSS:
    cleaned = html.strip()
    cleaned = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", " ", cleaned)
    
    # Then we remove html comments. This has to be done before removing regular
    # tags since comments can contain '>' characters.
    cleaned = re.sub(r"(?s)<!--(.*?)-->[\n]?", " ", cleaned)
    
    # Next we can remove the remaining tags:
    cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)   
    
    return cleaned.strip()

# Clean the HTML content
data_train['cleaned_text'] = data_train['text'].apply(clean_html)

# Display cleaned text examples
print("\n--- Cleaned text:---" )
print(data_train['cleaned_text'].head())



--- Cleaned text:---
0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
1                                             Will do.
2    Nora--Cheryl has emailed dozens of memos about...
3    Dear Sir=2FMadam=2C I know that this proposal ...
4                                                  fyi
Name: cleaned_text, dtype: object


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [10]:
# Your code
# Function to clean the text
def pre_process(text):
    # Convert to lowercase
    text = text.lower()

    # Remove all special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove all single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^[a-zA-Z]\s', '', text)
    
    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove prefixed 'b'
    text = re.sub(r'\bb\s', '', text)
    
    return text

# Cleaning the text
data_train['preprocessed_text'] = data_train['cleaned_text'].apply(pre_process)

# Display preprocessed text examples
print("\n--- Pre-processed text:---")
print(data_train['preprocessed_text'].head())


--- Pre-processed text:---
0    dear sir strictly private business proposal am...
1                                              will do
2    noracheryl has emailed dozens of memos about h...
3    dear sirfmadamc know that this proposal might ...
4                                                  fyi
Name: preprocessed_text, dtype: object


## Now let's work on removing stopwords
Remove the stopwords.

In [11]:
# Your code
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    filtered_words = []
    words = text.split()
    for word in words:
        if word not in stop_words:
            filtered_words.append(word)
    return ' '.join(filtered_words)

#removing stopwords
data_train['clean_text_no_stopwords'] = data_train['preprocessed_text'].apply(remove_stopwords)

# Display text after removing stopwords
print("\n--- Cleaned text without stopwords:---")
print(data_train['clean_text_no_stopwords'].head())


--- Cleaned text without stopwords:---
0    dear sir strictly private business proposal mi...
1                                                     
2    noracheryl emailed dozens memos haiti weekend ...
3    dear sirfmadamc know proposal might surprise e...
4                                                  fyi
Name: clean_text_no_stopwords, dtype: object


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [12]:
# Your code
#Lemmatization
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)        

data_train['clean_text_lemmatized'] = data_train['clean_text_no_stopwords'].apply(lemmatize_text)

print("\n--- Cleaned and lemmatized text:---")
print(data_train['clean_text_lemmatized'].head())


--- Cleaned and lemmatized text:---
0    dear sir strictly private business proposal mi...
1                                                     
2    noracheryl emailed dozen memo haiti weekend pl...
3    dear sirfmadamc know proposal might surprise e...
4                                                  fyi
Name: clean_text_lemmatized, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [18]:
# Your code
# Split the data into ham and spam
ham_messages = data_train[data_train['label']=='ham']['clean_text_lemmatized']
spam_messages = data_train[data_train['label']=='spam']['clean_text_lemmatized']
ham_spam_messages = pd.concat([ham_messages, spam_messages])



## Extra features

In [19]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

NameError: name 'data_val' is not defined

## How would work the Bag of Words with Count Vectorizer concept?

In [23]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer
vectorizer = CountVectorizer(stop_words='english')


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [24]:
# Your code
# TfidfVectorizer
vectorizer = TfidfVectorizer()
ham_spam_matrix = vectorizer.fit_transform(ham_spam_messages)

#Combined matrix back into ham and spam
ham_matrix = ham_spam_matrix[:len(ham_messages)]
spam_matrix = ham_spam_matrix[:len(ham_messages)]



ValueError: empty vocabulary; perhaps the documents only contain stop words

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code